# ECS 171 Group 1
## Demo Program with GUI
To run the program, there are three things to do:
1. Go to Spotify API website and apply for a private id and secret, put them into program
2. Download the model we use for this program, link is avaiable on GitHub README (https://drive.google.com/file/d/1kPJB3eIdsqBRgjUPaiEGTVTqy-zRRs6l/view?usp=sharing)
3. Make sure spotify_api.py and bg.gif is on same directory.

In [3]:
from tkinter import *
from tkinter import font as tkFont  # for convenience
import spotify_api
import requests
import pprint
import json
import io
from PIL import ImageTk, Image
import numpy as np
from urllib.request import urlopen
import base64
import pandas as pd
import joblib
from numpy import array

nn_model = joblib.load('random_forest_model200.joblib')

root = Tk()
root.title('Spotify Song Popularity')
root.geometry("1280x720")

# Maybe you need to change the image name here
bg = PhotoImage(file = "bg.gif",master = root)

my_canvas = Canvas(root,width=1200,height = 750)
my_canvas.pack(fill="both",expand = True)
my_canvas.create_image(0,0,image = bg, anchor="nw")

# Add Label
    
# Input Entry Bar
entry = Entry(my_canvas,bg = "#02de5a",width=30, font = ('Arial','20','bold'), bd = 0, justify='center')
entry_window = my_canvas.create_window(430,347,window=entry)


# Output Entry Bar
output = Entry(my_canvas,bg = "#2c4ace",fg = "#02de5a",width=12, font = ('Arial','25','bold'), bd = 0, justify='center')
output_window = my_canvas.create_window(1110, 545,window=output)

output2 = Entry(my_canvas,bg = "#2c4ace",fg = "#02de5a",width=12, font = ('Arial','25','bold'), bd = 0, justify='center')
output2_window = my_canvas.create_window(1045, 175,window=output2)



# Artist Name Bar
artist = Entry(my_canvas,bg = "#02de5a",fg = "#222222",width=30, font = ('Arial','20','bold'), bd = 0, justify='center')
artist_window = my_canvas.create_window(430, 487,window=artist)
 
# The function, which we should change to connect API and our model
def setTextInput():
    text = entry.get()

    # create your own client_id and client_secret tokens and plug in variables as strings
    # https://cran.r-project.org/web/packages/spotidy/vignettes/Connecting-with-the-Spotify-API.html
    client_id = ""
    client_secret = ""

    spotify = spotify_api.SpotifyAPI(client_id, client_secret)
    track = spotify.search(text, search_type = "track", limit = 1)
    track_items = track["tracks"]["items"]
    track_id = track_items[0]["id"]
    artist_json = track["tracks"]["items"]
    artist_name = artist_json[0]["artists"][0]["name"]
    real_pop = track['tracks']['items'][0]['popularity']
    real_pop = ((real_pop-1)//20)+1
    image_json = track["tracks"]["items"][0]["album"]["images"]
    image_url = image_json[0]["url"]
    year = track["tracks"]["items"][0]["album"]["release_date"]
    music_year = int(year[0:4])
    
    f = open('Album_photo.gif','wb')
    response = requests.get(image_url)
    f.write(response.content)
    f.close()
    
    img_open = Image.open("Album_photo.gif")
    img_open = img_open.resize((250, 250), Image.ANTIALIAS)
    album_img = ImageTk.PhotoImage(img_open)
    my_canvas.create_image(952, 236, anchor = NW, image = album_img)

    analysis_result = spotify.features(_id = track_id)
    #danceability  instrumentalness  liveness  loudness  speechiness
    danceability = analysis_result["danceability"]
    instrumentalness = analysis_result["instrumentalness"]
    liveness = analysis_result["liveness"]
    loudness = analysis_result["loudness"]
    speechiness = analysis_result["speechiness"]
    valence = analysis_result["valence"]

    data = {'danceability':[danceability], 'instrumentalness':[instrumentalness], 'liveness':[liveness], 'loudness':[loudness], 'speechiness':[speechiness], 'valence':[valence], 'year':[music_year]}
    df = pd.DataFrame(data)

    
    real_str = str(real_pop)
    if real_pop == 5:
        real_str = "★★★★★"
    if real_pop == 4:
        real_str = "★★★★"
    if real_pop == 3:
        real_str = "★ ★ ★"
    if real_pop == 2:
        real_str = "★  ★"
    if real_pop == 1:
        real_str = "★"

    y_pred = nn_model.predict(df)
    result = y_pred[0]
    result_str = str(result)
    
    if result == 5:
        result_str = "★★★★★"
    if result == 4:
        result_str = "★★★★"
    if result == 3:
        result_str = "★ ★ ★"
    if result == 2:
        result_str = "★  ★"
    if result == 1:
        result_str = "★"
    output.delete(0,"end")
    output.insert(0, real_str)
    output2.delete(0,"end")
    output2.insert(0, result_str)
    artist.delete(0,"end")
    artist.insert(0, artist_name)
    mainloop()
    
    
# Add buttons    
go_button = Button(root, text = "GO",fg = "#02de5a", width=7, height=1, command=lambda:setTextInput()) #When hit the button, run setTextInput(). We can change this function to API. 
helv36 = tkFont.Font(family='Allumi Std',weight='bold')
go_button['font'] = helv36
go_button.configure(bg = "#2c4ace")
button_window = my_canvas.create_window(797, 295, window=go_button)


root.mainloop()